In [1]:
from pathlib import Path
import os
from docling_optimize.config.models import Config
from pydantic import BaseModel, ValidationError
import yaml
import logging

from docling.document_converter import DocumentConverter, PdfFormatOption
from docling.datamodel.base_models import InputFormat
from docling.pipeline.standard_pdf_pipeline import StandardPdfPipeline
from docling.datamodel.pipeline_options import (
    PdfPipelineOptions
)
from docling_core.transforms.serializer.markdown import MarkdownDocSerializer

from docling_optimize.pymupdf_backend import PyMuPDFDocumentBackend

logger = logging.getLogger("DoclingExtractor")

c:\Users\fernjoh\Documents\projects\docling_optimize\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def _load_config(config_path: str | os.PathLike) -> Config:
    """Load configurations file."""
    path = Path(config_path)
    try:
        with path.open("r") as file:
            data = yaml.safe_load(file)
        return Config(**data)
    except (FileNotFoundError, ValidationError) as e:
        logger.error(f"Configuration error: {e}")
        raise


config_path = "../src/docling_optimize/config/app_config.yml"


config = _load_config(config_path)

converter = DocumentConverter(
    format_options={

        InputFormat.PDF: PdfFormatOption(
            pipline_cls=StandardPdfPipeline, 
            pipeline_options=PdfPipelineOptions(artifacts_path=Path(config.models.docling_path),
                                                                    do_ocr=False, 
                                                                    force_backend_text=True,
                                                                    # accelerator_options=accelerator_options
                                                                    do_table_structure = True,
                                                                    generate_picture_images=True
                                                                    ),
            # backend = DoclingParseV4DocumentBackend
            backend = PyMuPDFDocumentBackend
        ),
    }
)

In [3]:
filepath="../data/pdf/inputs/example_1.pdf"

doc = converter.convert(source=Path(filepath)).document

2026-01-09 16:03:14,968 - INFO - detected formats: [<InputFormat.PDF: 'pdf'>]
2026-01-09 16:03:14,990 - INFO - Parsing the document with PyMuPDF
2026-01-09 16:03:15,046 - INFO - Going to convert document batch...
2026-01-09 16:03:15,053 - INFO - Initializing pipeline for StandardPdfPipeline with options hash 4510ea0cf5f249bba7e1afb43d879eef
2026-01-09 16:03:15,182 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:03:15,193 - INFO - Registered picture descriptions: ['vlm', 'api']
2026-01-09 16:03:15,266 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:03:15,287 - INFO - Registered ocr engines: ['auto', 'easyocr', 'ocrmac', 'rapidocr', 'tesserocr', 'tesseract']
2026-01-09 16:03:15,624 - INFO - Loading plugin 'docling_defaults'
2026-01-09 16:03:15,648 - INFO - Registered layout engines: ['docling_layout_default', 'docling_experimental_table_crops_layout']
2026-01-09 16:03:15,762 - INFO - Accelerator device: 'cpu'
2026-01-09 16:03:19,794 - INFO - Loading plugin 'docling_defa

In [23]:
# doc.texts[0].label.value

len(doc.texts)

exclusion_texts = ["page_footer"]

included_text = []
for text_item in doc.texts:
    if text_item.label.value not in exclusion_texts:
        included_text.append(text_item)

len(included_text)

290

In [5]:
import os
import multiprocessing

# Method 1: Get total CPU cores (logical processors)
total_cores = multiprocessing.cpu_count()
print(f"Total CPU cores: {total_cores}")

# Method 2: Get physical CPU cores (more conservative)
try:
    physical_cores = len(os.sched_getaffinity(0))  # On Linux
    print(f"Available CPU cores: {physical_cores}")
except AttributeError:
    # Fallback for non-Linux systems
    physical_cores = os.cpu_count() or total_cores
    print(f"CPU cores (fallback): {physical_cores}")

# Method 3: Check current OMP_NUM_THREADS setting
current_omp_threads = os.getenv('OMP_NUM_THREADS')
print(f"Current OMP_NUM_THREADS: {current_omp_threads}")

Total CPU cores: 2
CPU cores (fallback): 2
Current OMP_NUM_THREADS: None
